In [72]:
import numpy.random as rand
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [73]:
df = pd.DataFrame(rand.random((500, 3)) * 1000, columns=['X', 'Y', 'Z'])
df['Class'] = rand.randint(low=1, high=3, size=500)
print(df)

              X           Y           Z  Class
0    247.624448  668.353523  821.259842      1
1    385.229367  343.281080  588.168449      2
2    476.712254  511.870447  216.102605      1
3    645.248974  836.596008  951.514103      1
4    484.879174  714.086799  334.152606      2
..          ...         ...         ...    ...
495  550.890517  653.760026  786.438989      2
496    5.417033  619.660765   40.155792      2
497  493.183663  125.374992  611.134942      2
498  809.765968  826.205499  266.681294      2
499  400.016747   47.510628  518.019356      2

[500 rows x 4 columns]


# Test and Train Set Creation

In [74]:
# X is feature set, y is label set
X = df.iloc[:, :3].values
y = df.iloc[:, 3].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Proprocessing and Scaling

In [75]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Linear Discriminant Analysis

In [76]:
num_components = 2
lda = LDA(n_components=num_components)
x = lda.fit_transform(X_train, y_train)

x = pd.DataFrame(x, columns=[f'LD{i+1}' for i in range(num_components)])
print(x.shape)

ValueError: n_components cannot be larger than min(n_features, n_classes - 1).

In [ ]:
plt.figure(figsize=(10, 10))
scatter = plt.scatter(x['LD1'], x['LD2'], c=y_train, cmap='rainbow')
plt.title('LDA on UCI Wine Dataset')
plt.xlabel('LD1')
plt.ylabel('LD2')